### Importing all the libraries

In [ ]:
# Importing all the libraries
import  boto3  
client = boto3.client('quicksight')
account_id = boto3.client('sts').get_caller_identity()['Account'] #extracting AWS account_ID

### The below function will create template from the available analysis. In our case we create two template one for Scoring analysis and other for drift analysis.

In [4]:
# FUNCTION TO CREATE TEMPLATE

def creating_template_from_analysis(account_id, Template_Id, template_name, analysis_arn, dataset_ref ):
    response = client.create_template(
    AwsAccountId=account_id,
    TemplateId=Template_Id,
    Name=template_name,
    SourceEntity={
        'SourceAnalysis': {
            'Arn': analysis_arn,
            'DataSetReferences':dataset_ref,
        } })
    return response


### The List API will fetch list of all the quicksight resources in the current account. We have to copy the Id of each datasource, dataset, analysis/dashboard resources that we want to replicate in the other account.

In [7]:
list_datasource = client.list_data_sources(AwsAccountId= account_id)
list_dataset = client.list_data_sets(AwsAccountId=account_id)
list_analysis = client.list_analyses(AwsAccountId=account_id)
# list_template = client.list_templates(AwsAccountId=account_id)
# list_dashboard = client.list_dashboards(AwsAccountId=account_id)

### Provide the resource id. Remember we have to create template from the analysis and so we have to assign a template ID, template name and Placeholder_list [ place holder is used to map a dataset with a name while creating a template].

In [ ]:
DataSourceID = '44da590f-5cf4-4a18-9e46-042187db9451'
DataSetID = '4a3ce89b-c7c5-4c11-a121-d29df554ec3d'
TemplateID = 'TempalteID123123123'
TemplateName= 'TemplateName'
placeholder_list=['p11']
AnalysisID = '011d8c63-980d-4a6a-be28-2b1756ea72f5'
Account_B = ""  #'Provide Prod account ID here'

### We are fetching data ARN list and analysis ARN of our analysis resource which can be found in output of describe_analysis API.

In [ ]:
#EXTRACTING INFO REQUIRED FOR CREATING TEMPLATE
des_analysis = client.describe_analysis(AwsAccountId=account_id,
    AnalysisId=AnalysisID)
dataset_arn_list = des_analysis['Analysis']['DataSetArns']
AnalysisARN = des_analysis['Analysis']['Arn']

#### Mapping each dataset_arn with placeholder which will be used in creating a template. Providing argument to template creating function  and creating the template. 
#### Once the template is created it is shared to the other AWS account(INT) account by calling client.update_template_permissions() this function.

In [ ]:
# Creating_template:-
def creating_template_from_analysis(account_id, Template_Id, template_name, analysis_arn, dataset_ref ):
    response = client.create_template(
    AwsAccountId=account_id,
    TemplateId=Template_Id,
    Name=template_name,
    SourceEntity={
        'SourceAnalysis': {
            'Arn': analysis_arn,
            'DataSetReferences':dataset_ref,
        } })
    return response

#resource is having one ARN so 1 placeholder is given/ Dataset used for the analysis and Placeholder should be in same number
dataset_ref = list()
for i in range(len(dataset_arn_list)):
    single_datset_ref = {'DataSetArn': dataset_arn_list[i], 'DataSetPlaceholder': placeholder_list[i]}
    dataset_ref.append(single_datset_ref)
dataset_ref
print(dataset_ref)

#Creating template here
res = creating_template_from_analysis(account_id, TemplateID, TemplateName, AnalysisARN, dataset_ref)
print(res)

#Sharing the template in the Prod env directly

response_updated_perm = client.update_template_permissions(
    AwsAccountId=account_id,
    TemplateId=TemplateID,
    GrantPermissions=[
        {
            'Principal': f'arn:aws:iam::{Account_B}:root',  #PUt here INT Env AccountID
            'Actions': ['quicksight:UpdateTemplatePermissions', 'quicksight:DescribeTemplatePermissions', 'quicksight:UpdateTemplateAlias', 'quicksight:DeleteTemplateAlias', 'quicksight:DescribeTemplateAlias', 'quicksight:ListTemplateAliases', 'quicksight:ListTemplates', 'quicksight:CreateTemplateAlias', 'quicksight:DeleteTemplate', 'quicksight:UpdateTemplate', 'quicksight:ListTemplateVersions', 'quicksight:DescribeTemplate', 'quicksight:CreateTemplate']
        },
    ]
)
print(response_updated_perm)



### Creating Json file in a structured format by describing datasource, dataset, template and analysis/dashboard. The dataset and datasource json files has a specific format. Do follow the structure as this files will be used in a similar way while being created/Replicated in the PROD env.

### Below cells will automatically create the json files which we need for creating the resoureces in the INT env. We just need to take care of mapping datasource with the dataset. Always start the naming conventions from datasource_0, datasource_1 and dataset_0,dataset_1 and keep increasing the count if more than one datasource or dataset is used in your analysis. 


In [1]:
#creating json file for all reosurce

# Creating Datasource json file with proper format
describe_datasource = client.describe_data_source(AwsAccountId=account_id,DataSourceId=DataSourceID)

#Creating a json file
with open('datasource.json', 'w' ) as out:
    json.dump(describe_datasource, out , default=str)

#describing dataset and its format
describe_dataset = client.describe_data_set(AwsAccountId= account_id,  DataSetId=DataSetID)
with open('datset.json', 'w' ) as out:
    json.dump(describe_dataset, out , default=str)
# Describing template
describe_template =  client.describe_template(AwsAccountId=account_id,TemplateId=TemplateID)
with open('template.json', 'w' ) as out:
    json.dump(describe_template, out , default=str)

#this is used in describe analysis Code
describe_analysis = {"analysis_name": "ScoringAnlaysis", #Giving anlaysis name
"analysis_id":"ScoringAnlaysis", #giving analysis ID
"data_mapping":[{"dataset_0":"datasource_0"}], #Mapping which dataset belong to which datasource  ex:-{"dataset_1":"datasource_1"}
"list_folder":["VCI-DataLake/VCI-MLOps/VCI-UserCarModel/Dashboard","VCI-DataLake/VCI-MLOps/VCI-UserCarModel/Assets"]
}  #folder list is here. QS resources are added into this folders. Inside Assets dataset and analysis is added, inside dashboard folder dashboards are added.

with open('analysis.json', 'w' ) as out:
    json.dump(describe_analysis, out , default=str)

#this is used in describe dashboard Code
describe_dashboard = {"dashboard_name": "Scoringdashboard", #dahsboard name
"dashboard_id":"Scoringdashboard",  #dashboard ID providing
"data_mapping":[{"dataset_0":"datasource_0"}], #,{"dataset_1":"datasource_1"} #mapping which datasource belongs to which dataset.
"list_folder":["VCI-DataLake/VCI-MLOps/VCI-UserCarModel/Dashboard","VCI-DataLake/VCI-MLOps/VCI-UserCarModel/Assets"]}
with open('dashboard.json', 'w' ) as out:
    json.dump(describe_dashboard, out , default=str)
    


NameError: name 'quicksight' is not defined

##  Update Template codes:-
#### This section should be used to create json files for updating any current resource such as dataset, datsource or templates. Suppose our current dataset is updated with some new columns in the dev env. So we'll have to replicate the same in the INT, for that we have to get the updated description of that resource and use it to update the INT env Resource.

In [ ]:
#creating json file for all reosurce

# Creating Datasource json file with proper format
describe_datasource = client.describe_data_source(AwsAccountId=account_id,DataSourceId=DataSourceID)

with open('datasource.json', 'w' ) as out:
    json.dump(describe_datasource, out , default=str)

In [ ]:
#describing dataset and its format

describe_dataset = client.describe_data_set(AwsAccountId= account_id,  DataSetId=DataSetID)
with open('datset.json', 'w' ) as out:
    json.dump(describe_dataset, out , default=str)

In [ ]:
# Template either for analysis
describe_template = {describe_template,
"action":"analysis",
"analysis":{"analysis_id":"ScoringAnlaysisLambdaTrial4","analysis_name":"ScoringAnlaysisLambdaTrial4"}}

# Describing template
describe_template =  client.describe_template(AwsAccountId=account_id,TemplateId=TemplateID)
with open('template.json', 'w' ) as out:
    json.dump(describe_template, out , default=str)

In [ ]:
# Template either for dashboard
describe_template = {describe_template,
"action":"dashboard",
"dashboard":{"dashboard_id":"Scoringdashboard","dashboard_name":"Scoringdashboard"}}

# Describing template
describe_template =  client.describe_template(AwsAccountId=account_id,TemplateId=TemplateID)
with open('template.json', 'w' ) as out:
    json.dump(describe_template, out , default=str)